In [ ]:
#| default_exp microclip

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
import subprocess
from fastai.vision.all import *
from multiprocessing import Pool

In [ ]:
#| export
def create_microclips(dataroot_path, offset=120, fps=1, scale_factor=0, n_workers=4):
    """
    Create microclips (small mp4 files) from frames.

    Args:
    - `dataroot_path` (`str` or `Path`): Root path of the data directory.
    - `offset` (`int`): Number of seconds of video until the anchor frame.
    - `fps` (`int`): Frames per second for the extracted microclip.
    - `scale_factor` (`int`): Factor to scale the microclip dimensions.
        - `2` means half the height and width.
        - `0` or `-1` means original size.
    - `n_workers` (`int`): Number of worker processes to use.

    Returns:
    - `None`
    """
    dataroot_path = Path(dataroot_path)
    frames_folder = dataroot_path / 'frames'
    microclips_dir = dataroot_path / 'microclips'
    microclips_dir.mkdir(parents=True, exist_ok=True)
    temp_dir = dataroot_path / 'temp'
    temp_dir.mkdir(parents=True, exist_ok=True)

    frames = get_image_files(frames_folder)
    data = []
    for frame_path in frames:
        filename = frame_path.name
        surgery_name, video_name, timestamp = parse_filename(filename)
        data.append({
            'frame_path': frame_path,
            'surgery_name': surgery_name,
            'video_name': video_name,
            'timestamp': timestamp
        })

    # Prepare arguments for parallel processing
    args = [
        (
            item,
            microclips_dir,
            dataroot_path,
            offset,
            fps,
            temp_dir,
            scale_factor
        )
        for item in data
    ]

    # Use multiprocessing Pool to process microclips in parallel
    with Pool(processes=n_workers) as pool:
        pool.starmap(process_microclip, args)

In [ ]:
#| export
def process_microclip(item, microclips_dir, dataroot_path, offset, fps, temp_dir, scale_factor):
    """
    Process a single microclip.

    Args:
    - `item` (`dict`): Dictionary containing frame info.
    - `microclips_dir` (`Path`): Directory to save microclips.
    - `dataroot_path` (`Path`): Root path of the data directory.
    - `offset` (`int`): Number of seconds of video until the anchor frame.
    - `fps` (`int`): Frames per second for the extracted microclip.
    - `temp_dir` (`Path`): Directory for temporary files.
    - `scale_factor` (`int`): Factor to scale the microclip dimensions.

    Returns:
    - `None`
    """
    frame_path = item['frame_path']
    filename = frame_path.name
    surgery_name = item['surgery_name']
    video_name = item['video_name']
    timestamp = item['timestamp']

    microclip_name = frame_path.stem + '.mp4'
    microclip_path = microclips_dir / microclip_name

    video_path = dataroot_path / 'videos' / surgery_name / f'{video_name}.mp4'

    if not video_path.exists():
        print(f"Video file '{video_path}' does not exist.")
        return

    start_time = max(0, timestamp - offset)
    duration = offset  # Duration is the offset since we want the clip leading up to the timestamp

    if duration <= 0:
        print(f"Invalid duration for {video_path}: start_time={start_time}, duration={duration}")
        return

    temp_video_path = temp_dir / f'{frame_path.stem}_temp.mp4'

    extract_clip(video_path, start_time, duration, temp_video_path, fps, scale_factor)

    try:
        temp_video_path.replace(microclip_path)
        print(f"Successfully created microclip: {microclip_path}")
    except Exception as e:
        print(f"Error moving {temp_video_path} to {microclip_path}: {e}")

In [ ]:
#| export
def extract_clip(video_path, start_time, duration, output_path, fps, scale_factor):
    """
    Extract a clip from the video.

    Args:
    - `video_path` (`Path`): Path to the source video.
    - `start_time` (`float`): Start time in seconds.
    - `duration` (`float`): Duration in seconds.
    - `output_path` (`Path`): Path to save the extracted clip.
    - `fps` (`int`): Frames per second for the extracted clip.
    - `scale_factor` (`int`): Factor to scale the microclip dimensions.

    Returns:
    - `None`
    """
    # Base FFmpeg command
    ffmpeg_cmd = [
        'ffmpeg',
        '-ss', str(start_time),
        '-i', str(video_path),
        '-t', str(duration),
        '-vf', f'fps={fps}',
        '-c:v', 'libx264',
        '-an',  # No audio
        '-y',  # Overwrite output files without asking
        str(output_path),
        '-hide_banner',
        '-loglevel', 'error'
    ]

    # If scale_factor is greater than 1, add scaling to the video filter
    if scale_factor > 1:
        # Calculate new width and height based on the scale factor
        # FFmpeg allows dynamic scaling using expressions
        scale_filter = f'scale=iw/{scale_factor}:ih/{scale_factor}'
        # Combine fps and scale filters
        ffmpeg_cmd[8] = f'fps={fps}, {scale_filter}'  # Update the -vf parameter

    result = subprocess.run(ffmpeg_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    if result.returncode != 0:
        print(f"Failed to extract clip from {video_path} to {output_path}")
        print(f"FFmpeg Error: {result.stderr}")

In [ ]:
#| export
def parse_filename(filename):
    """
    Parse the filename to extract surgery name, video name, and timestamp.

    Args:
    - `filename` (`str`): Filename to parse.

    Returns:
    - `surgery_name` (`str`)
    - `video_name` (`str`)
    - `timestamp` (`float`)
    """
    base = Path(filename).stem
    parts = base.split('_')
    if len(parts) < 3:
        raise ValueError(f"Filename '{filename}' does not contain enough underscores.")
    surgery_name = parts[0]
    timestamp_str = parts[-1]
    try:
        timestamp = float(timestamp_str)
    except ValueError:
        raise ValueError(f"Expected a timestamp at the end of '{filename}', but got '{timestamp_str}'.")
    video_name = '_'.join(parts[1:-1])
    return surgery_name, video_name, timestamp

In [ ]:
#| export
# Example usage:
if __name__ == "__main__":
    dataroot_path = '../temset'  # Your actual data root path

    # Generate microclips with default size (original dimensions)
    create_microclips(
        dataroot_path=dataroot_path,
        offset=60,    # Number of seconds until the anchor frame
        fps=1,        # Frames per second for the extracted microclip
        scale_factor=0,  # No scaling, original size
        n_workers=4    # Number of worker processes to use
    )

    # To generate microclips with scaled-down size (e.g., half the size)
    create_microclips(
        dataroot_path=dataroot_path,
        offset=60,
        fps=1,
        scale_factor=4,  # Scales the video to half the width and height
        n_workers=1
    )

In [23]:
#| hide
import nbdev; nbdev.nbdev_export()

SyntaxError: invalid syntax (<unknown>, line 1)